1.keras.preprocessing.text.Tokenizer() to transform text to index 


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from collections import Counter
import re
 
# nltk
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
 
# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder



---

Sentiment classification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# movie = pd.read_csv('/content/drive/MyDrive/00NLP/movie_data.csv', encoding='utf-8')
train=pd.read_csv("/content/drive/MyDrive/yolo/kerasbert/train.csv").reset_index(drop=True)
test=pd.read_csv("/content/drive/MyDrive/yolo/kerasbert/test.csv").reset_index(drop=True)
print(train.shape, test.shape)

In [ ]:
train

In [ ]:
movie = pd.concat([train, test])
movie.shape

In [ ]:
movie.head()

In [ ]:
movie.tail()

In [ ]:
movie = movie.reset_index(drop=True)

In [ ]:
movie.tail()

In [ ]:
target_cnt = Counter(train.sentiment)

print(target_cnt.keys(), target_cnt.values())

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

In [ ]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) # re.sub 取代
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text) # :-) ;-( =-D :-P :D :-(
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text
 
def preprocess(text, stem=False):
    text = preprocessor(str(text).strip())
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [ ]:
movie.review = movie.review.apply(lambda x: preprocess(x))

In [ ]:
movie.head()

In [ ]:
movie.tail()

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(movie.review)
 
vocab_size = len(tokenizer.word_index) + 1 # pad 0
print("Total words", vocab_size)

In [ ]:
df = pd.DataFrame()

In [ ]:
df['review'] = tokenizer.texts_to_sequences(movie.review)

In [ ]:
df

In [ ]:
df['sentiment'] = movie['sentiment'] # 記得前面要做 movie.reset_index(drop=True)

In [ ]:
df

In [ ]:
# df_train, df_test = train_test_split(df, test_size=0.1, random_state=1)
# print("TRAIN size:", len(df_train))
# print("TEST size:", len(df_test))

In [ ]:
df_train = df[df['sentiment'].notnull()]

In [ ]:
df_test = df[df['sentiment'].isnull()] # nan

In [ ]:
df_test

In [ ]:
x_train = keras.preprocessing.sequence.pad_sequences(df_train.review.values, maxlen=512) # 512 ??
x_test = keras.preprocessing.sequence.pad_sequences(df_test.review.values, maxlen=512)

In [ ]:
x_test

In [ ]:
x_test.shape

In [ ]:
y_train = df_train.sentiment.values.reshape(-1,1)
# y_test = df_test.sentiment.values.reshape(-1,1)

print("y_train",y_train.shape)
# print("y_test",y_test.shape)

In [ ]:
# y_test

In [ ]:
max_features = vocab_size  # 要考慮作為特徵的語詞數量
maxlen = 512 # 當句子的長度超過300個語詞的部份,就把它刪除掉
batch_size = 256

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, GRU

model = Sequential()   # 輸入 （sample size，time steps） with label encoding              
model.add(Embedding(max_features, 16, input_length=maxlen)) # => 參數(input features, output features, time steps)
model.add(Dropout(0.5)) # 承接 embedding 輸出 （sample size，time steps, output features） 
# model.add(LSTM(32, return_sequences=True)) 
model.add(GRU(8))
model.add(Dropout(0.5))
# model.add(LSTM(4))
# model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/yolo/kerasbert/mov.h5', monitor='val_accuracy', mode="max", save_best_only=True, verbose=1)
earlystopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', mode="max", patience=5, verbose=1)
rlr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, mode='auto', min_delta=0.0001)

model.fit(x_train, y_train, epochs=30, batch_size=batch_size,
     validation_split=0.2, callbacks=[checkpoint,earlystopping,rlr])

In [ ]:
# model.evaluate(x_test, y_test, batch_size=16)

In [ ]:
test_pred = model.predict(x_test)

In [ ]:
test_pred

In [ ]:
result = pd.DataFrame()
result['ID'] = test['ID']
result['sentiment'] = test_pred.round().astype(int).reshape(-1)

In [ ]:
result

In [ ]:
result.to_csv('/content/drive/MyDrive/yolo/kerasbert/submission.csv', index = False)